In [1]:
# imports
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import math
import keras
from scipy.stats  import norm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras        import backend as K
from datetime     import datetime
from keras.optimizers      import RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics       import mean_squared_error
from matplotlib.pylab      import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 15, 6

# number of inputs to be fed
look_back = 288
# number of outputs to be generated
look_forward = 24
# the scope of the LSTM Model
stations = 66

# convert an array of values into a dataset matrix, adjusted to make a dateset that is 66 wide
def create_dataset(dataset, look_back=1, look_forward=2):
    dataX, dataY = [], []
    np.array(dataY)
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back),:]
        # Makes sure that the array isn't the last 2 or 3 or whatever bits. It has to be the full 24
        if len(dataset[i + look_back:i+look_back+look_forward, 0]) == look_forward:
            dataX.append(a.T)
            dataY.append(dataset[i + look_back:i+look_back+look_forward, :].T)
    return np.array(dataX), np.array(dataY)

def predict_with_uncertainty(model, dataset, n_iter=100):
    # This function activates drop-out when doing predictions
    f = K.function([model.layers[0].input, K.learning_phase()],
                      [model.layers[-1].output])
    all_pred = np.zeros((n_iter,) + (stations,look_forward,) )

    for iter in range(n_iter):
        all_pred[iter] = np.array( f([dataset.reshape( (1,) + dataset.shape), 1]) ).reshape(66, 24)
    avg_pred = all_pred.mean(axis=0)
    std = np.std(all_pred, axis=0)
    return all_pred, avg_pred, std

# given a model and data from the original dataset, it uses predict_with_uncertainty to predict with dropout
def run_predictions (model, dataset):

    # scale the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(np.array(dataset))
    data = dataset.T

    # feeds the model and data into a function that will activate the dro-out
    all_pred, avg_pred, std = predict_with_uncertainty(model, data)

    # All the predictions
    for i in range(stations):
        all_pred[:,i,:] = scaler.inverse_transform(all_pred[:,i,:])

    # The Average Prediction
    avg_pred = scaler.inverse_transform(avg_pred)
    # The Standard Deviation At Each T Of Prediction
    std = scaler.inverse_transform(std)

    return all_pred, avg_pred, st

Using TensorFlow backend.


In [3]:
def predict (model, day=11, total_days=1, delta=1, show_gradual_error=False, show_graph=True):
    # creates matrices to hold the 
    avg_preds = np.zeros((288*total_days/delta, look_forward))
    true_vals = np.zeros((288*total_days/delta, look_forward))
    cdf_vals  = np.zeros((288*total_days/delta, look_forward))

    percent_over = np.zeros(288*total_days/delta)
    # Runs all the predictions
    for i in range (288*total_days/delta):
        # the point where the model will predict
        point = 288*day + i * delta

        # Gets predictions
        all_pred, avg_pred, std = run_predictions (model, dataset_orig[point-look_back:point])

        # Reshapes all of the things
        true_val = dataset_orig[point:point+look_forward].reshape(look_forward)
        avg_pred = avg_pred.reshape(look_forward)
        std = std.reshape(look_forward)

        percent_over[i] = 100.0 * (np.ones((len(all_pred),1) )*true_val - all_pred < 0).sum() / all_pred.size

        # converts the original values to cdf values according to the cumullative distibution functions at t+1, t+2, ..., t+24 (0.0-1.0)
        # Adds the values to the overall matrix of cdf
        cdf_vals[i,:] = norm.cdf(true_val, loc=avg_pred, scale=std)

        avg_preds[i], true_vals[i] = avg_pred, true_val

        if(show_graph):
            # plots the uncertainty to the degree of half a standard deviation
            plt.fill_between(range(point, point+look_forward), 
                             avg_pred + std/2.0, 
                             avg_pred - std/2.0, 
                             facecolor='red', alpha=0.25)
            # plots the uncertainty to the degree of a full standard deviation
            plt.fill_between(range(point, point+look_forward), 
                             avg_pred + std, 
                             avg_pred - std, 
                             facecolor='red', alpha=0.25)
            # plots the average predictions
            plt.plot(range(point, point+look_forward), avg_pred, color='red', alpha=1)

    print "Percent Overestimating: {}".format(np.mean(percent_over))

    if show_graph:
        plt.plot( np.arange(288*total_days+look_forward) + 288 * day, dataset_orig[288*day:288*day+288*total_days + look_forward,0])
        plt.axhline(0)
        plt.show()

    if show_gradual_error:
        indiv_err = true_vals - avg_preds
        for i in range(look_forward):
            plt.scatter(np.ones(len(indiv_err[:,i]))*i,indiv_err[:,i], color='black',alpha=0.1)
        plt.axhline(0)
        plt.show()
    return cdf_v


In [4]:
# The interval between each dataset (original data in 5 minute intervals)
time_grouping = '5min'

# load the dataset
dataframe = pd.read_csv('ignored_assets/paxout_table.csv', engine='python', nrows=288*21)
dataframe['time_bucket'] = pd.to_datetime(dataframe['time_bucket'])
dataframe = dataframe.set_index('time_bucket')
# dataframe['total'] = dataframe.sum(axis=1)
dataframe['day_hour'] = dataframe.index.round(time_grouping)
dataframe = dataframe.groupby('day_hour').sum()
# removes the timestamp at column 67
dataset_orig = dataframe.values[:,:stations]
dataset_orig = dataset_orig.astype('float32')

# scale the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset_orig)


In [6]:
# split into train and test sets
train_size = 288 * 10
test_size = len(dataset) - train_size
test, train = dataset[0:test_size,:], dataset[test_size:len(dataset)-look_forward,:]

# reshape into X=[t, t-1, t-2,..., t-look_back] and Y=[t+1, t+2,... t+look_forward]
trainX, trainY = create_dataset(train, look_back, look_forward)
testX, testY = create_dataset(test, look_back, look_forward)
# reshape input to be [samples, time steps, features]
print trainX.shape


(2545, 66, 288)


In [10]:
# creates a model but does not compile the model
def create_model(rate = 0.2):
    model = Sequential()
    model.add(Dropout(rate, input_shape=(stations, look_back)))
    lstm = LSTM(256, recurrent_dropout=0.3, return_sequences=True)
    model.add(lstm)
    model.add(Dropout(rate))
    model.add(Dense(128))
    model.add(Dropout(rate))
    model.add(Dense(128))
    model.add(Dropout(rate))
    model.add(Dense(128))
    model.add(Dense(look_forward))
    return model

In [11]:
def priorities (y_true, y_pred):
    w = np.arange(1,0, -1./24)**1.3
    w = w / w.sum()
    w = w[:,None]
    W = K.variable(value = w)
    return K.dot( K.abs(y_pred-y_true), W)

batch = 1
seed = 11
# create and fit the LSTM network
model = create_model(rate=0.2)

np.random.seed(seed)
model.compile(loss=priorities, optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0))

for i in range(1):
    model.fit(trainX, trainY, epochs=60, batch_size=60, verbose=2)

predict(model, total_days=1,day=11,delta=24)

# cdf_matrix = predict(model, total_days=1,day=11,delta=1, show_graph=False)
# for i in range (24):
#     plt.hist(cdf_matrix[:,i],20)
#     plt.show()

Epoch 1/60


KeyboardInterrupt: 

In [ ]:
predict(model, total_days=1,day=11,delta=24)